# Process RR_peaks to metrics

**The following notebook** 
* Reads the RR peaks file
* Derives metrics from 5 min intervals
* Joins clinical indicator file with metrics on patient id 
* Saves the file

In [ ]:
import pandas as pd
import h5py
import helper

read_path = "our_processing\ML_Data.h5"
pickle_path = 'our_processing/Waj_05_02_2025.pkl'

Read the RR_Peaks file ["our_processing\ML_Data.h5"]("our_processing\ML_Data.h5")

In [ ]:


def read_data(file_path, group_name, signal_name= "filtered_intervals"):
    with h5py.File(file_path, "r") as hdf:
        group = hdf[group_name]
        data = group[signal_name][()]
    return data

group_names = [
"19070921", "19072205", "19072214",
"19072938", "19072939", "19072940",
"19080106", "19080715", "19081506",
"19082406", "19090308", "19090320",
"19101607", "19101619", "19102102",
"19102103", "19102524", "19102622",
"19112609", "19120302", "19120323",
"19120704", "19120723", "19121303",
"19121735", "20010826", "20010827",
"20011712", "20050628", "20052606",
"20061729", "20092226", "20092535",
"20101424", "20101822", "20102029",
"20120116", "20120922", "20121033",
"20121716", "20121718", "20122932", "20123017"]

patient_arrays = {group_name: read_data(read_path, group_name) for group_name in group_names}

patient_arrays

Calculate the metrics for each patient.

In [ ]:
metrics_df = pd.DataFrame()

for id, signal in patient_arrays.items():
    individual_metrics = helper.patients_metrics(signal)
    individual_metrics['id'] = id
    print(individual_metrics)
    metrics_df = pd.concat([metrics_df, individual_metrics], ignore_index=True)

Read the clinical indicators

In [ ]:
clin_indic = helper.df_from_excel('actionable_data/Clinical indicators.xlsx')
clin_indic.rename(columns={'number': 'id'}, inplace=True)
clin_indic['id'] = clin_indic['id'].astype(str)
display(clin_indic.columns)

Inner merge the clinical indicators and processing results

In [ ]:
metrics_df["id"].astype(str)
merged_data = metrics_df.merge(clin_indic, on='id', how='inner')

merged_data.head()

In [ ]:
merged_data.to_pickle(pickle_path)
merged_data.to_csv(pickle_path.replace(".pkl", '.csv'), index=False)